# Download Data and Running Lux Experiments
# =========================================

In this file we will:
1. Download the latest data from the Google Repository
2. Run Various Tests on the Data as defined
3. Produce plots that highlight the results of the tests

The notebook is designed to compare the Lux results from a variety of different emergency window replacement kits.

In [8]:
# Auto update notebook imports
#%load_ext autoreload
#%autoreload 2

# Backtrack to folder source directory if it doesn't already exist in path
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

sys.path.append(os.getcwd())
print(os.getcwd())

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from urllib.request import urlretrieve

DEVELOPMENT = True

raw_filepath = "data/raw/"
interim_filepath = "data/interim/"
external_filepath = "data/external/"
processed_filepath = "data/processed/"

/Users/harryhopalot/Documents/GitHub/dssd_Insulate_ukraine


In [19]:
# Filename 
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQb40H_HRBq-7bLcEHqEPB-qNep5Ls1a-Lha2rGnRKefccGXaphgTNvRg8LbA_uOB8Ws5sEo-G_Mn1Y/pub?gid=2106447875&single=true&output=csv"
filename = raw_filepath + "temperature_values.csv"
# Use url request to download a file from the internet
urlretrieve(url, filename)


('data/raw/temperature_values.csv', <http.client.HTTPMessage at 0x11fac7a10>)

In [45]:
# Read in the lux values
temp = pd.read_csv(filename, index_col=0, parse_dates=True)

 # Reduce the number of columns by taking the mean of the three sensors
columns = ["internal_temp", "external_temp", "internal_surface_temp", "external_surface_temp"]
for column in columns:
    temp[column] = temp[[column + "_1", column + "_2", column + "_3"]].mean(axis=1)
    temp = temp.drop([column + "_1", column + "_2", column + "_3"], axis=1)

temp.head()

/var/folders/yn/y0g2nf7d6tg_3p0ftb5f049c0000gn/T/ipykernel_25507/3593262376.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,window_type,internal_temp,external_temp,internal_surface_temp,external_surface_temp
house_id,,,,,
1,OSB,11.000000,6.666667,10.833333,7.333333
2,IU,20.666667,8.000000,17.666667,10.000000
3,UN,10.666667,3.000000,9.000000,3.500000
4,DOUBLE_GLAZED,8.666667,6.500000,11.000000,6.666667


In [46]:
# This code is here to allow me to ensure that plots we produce will suffice
if DEVELOPMENT:
    new_temp = temp.copy()
    # Repeat the following 30 times
    for i in range(29):
        # For each row in df
        for idx, row in temp.iterrows():
            window_type = row["window_type"]
            internal_temp = row["internal_temp"]
            external_temp = row["external_temp"]
            internal_surface_temp = row["internal_surface_temp"]
            external_surface_temp = row["external_surface_temp"]

            # Create a new row with the same window type and add some gaussian noise to the temperature values
            new_row = pd.DataFrame([[window_type, internal_temp + np.random.normal(), external_temp + np.random.normal(), internal_surface_temp + np.random.normal(), external_surface_temp + np.random.normal()]], columns=temp.columns)
            # Append the new row to the new_temp dataframe
            new_temp = pd.concat([new_temp, new_row], ignore_index=True)

    temp = new_temp

In [47]:
# For each row caluclate the difference between (internal_surface_temp - external_surface_temp)/(internal_temp - external_temp)
temp["delta_temp"] = (temp["internal_surface_temp"] - temp["external_surface_temp"])/(temp["internal_temp"] - temp["external_temp"])
temp.head(10)

,window_type,internal_temp,external_temp,internal_surface_temp,external_surface_temp,delta_temp
0,OSB,11.000000,6.666667,10.833333,7.333333,0.807692
1,IU,20.666667,8.000000,17.666667,10.000000,0.605263
2,UN,10.666667,3.000000,9.000000,3.500000,0.717391
3,DOUBLE_GLAZED,8.666667,6.500000,11.000000,6.666667,2.000000
4,OSB,9.604962,6.329379,11.604082,7.101668,1.374538
5,IU,20.208719,7.299661,19.815345,11.209884,0.666622
6,UN,11.192397,0.650434,8.434780,4.356504,0.386861
7,DOUBLE_GLAZED,9.194203,6.254086,11.674756,6.517331,1.754156
8,OSB,10.454432,6.238682,9.122608,5.367711,0.890683
9,IU,19.167626,6.056010,14.969834,9.644246,0.406173


In [52]:
# Create and plot descriptive statistics of the temperature deltas
temp_stats = temp.groupby("window_type").describe()["delta_temp"]
temp_stats.head()

# Use temp stats to plot a box plot of the temperature deltas of each window type
fig = px.box(temp, x=temp.window_type, y="delta_temp", title="Temperature Delta of Each Window Type")
fig.update_layout(xaxis_title="Window Type", yaxis_title="Temperature Delta")
fig.show()



In [27]:
# Calculate descriptive statistics for each of the lux values
temp.describe()


,House ID,internal_temp,external_temp,internal_surface_temp,external_surface_temp,delta_temp
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,2.500000,12.750000,6.041667,12.125000,6.875000,1.032587
std,1.290994,5.377422,2.136001,3.803933,2.671437,0.650236
min,1.000000,8.666667,3.000000,9.000000,3.500000,0.605263
25%,1.750000,10.166667,5.625000,10.375000,5.875000,0.689359
50%,2.500000,10.833333,6.583333,10.916667,7.000000,0.762542
75%,3.250000,13.416667,7.000000,12.666667,8.000000,1.105769
max,4.000000,20.666667,8.000000,17.666667,10.000000,2.000000
